In [2]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [3]:
from simpletransformers.ner import NERModel, NERArgs

mtype='xlmroberta'
model_args = NERArgs()
# model_args.num_train_epochs=20
model_args.labels_list = ['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim', 'I-per', 'I-gpe', 'I-tim']
# model_args.overwrite_output_dir = True
model = NERModel(mtype,r"/content/gdrive/MyDrive/Models/checkpoint-7803-epoch-9",args=model_args,use_cuda=False)

In [5]:
!pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import torch
from simpletransformers.ner import NERModel, NERArgs
from torch.profiler import profile, record_function, ProfilerActivity

# Define the input sentences
sentences = ["I am travelling to India and I work for Google", "I am a Microsoft employee"]

# Load the original model
mtype='xlmroberta'
model_args = NERArgs()
model_args.labels_list = ['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim', 'I-per', 'I-gpe', 'I-tim']
model = NERModel(mtype, r"/content/gdrive/MyDrive/Models/checkpoint-7803-epoch-9", args=model_args, use_cuda=False)

# Measure the time taken and memory consumed by the original model
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        predictions, raw_outputs = model.predict(sentences)

print("Original model:")
print(prof.key_averages().table(sort_by="self_cpu_time_total"))

# Quantize the model (not applicable for simpletransformers)
quantized_model = torch.quantization.quantize_dynamic(
    model.model, {torch.nn.Linear}, dtype=torch.qint8)
quantized_model = NERModel(mtype,model_name="/content/xlmroberta/checkpoint-7803-epoch-9", args=model_args, use_cuda=False)

#Measure the time taken and memory consumed by the quantized model (not applicable for simpletransformers)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        quantized_predictions, quantized_raw_outputs = quantized_model.predict(sentences)

print("Quantized model:")
print(prof.key_averages().table(sort_by="self_cpu_time_total"))


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Original model:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        59.63%        2.114s       100.00%        3.545s        3.545s             1  
                                            aten::addmm        35.67%        1.265s        36.02%        1.277s      17.494ms            73  
                                              aten::bmm         1.07%      37.832ms         1.07%      37.860ms       1.577ms            24  
                                             aten::gelu         0.96%      34.161ms         0.96%      34.161ms       2.847ms       

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Quantized model:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        50.45%        1.519s       100.00%        3.010s        3.010s             1  
                                            aten::addmm        44.26%        1.332s        44.78%        1.348s      18.464ms            73  
                                              aten::bmm         1.31%      39.520ms         1.31%      39.530ms       1.647ms            24  
                                             aten::gelu         1.10%      33.162ms         1.10%      33.162ms       2.764ms      